<a href="https://colab.research.google.com/github/byeongdon/hanghae99/blob/main/5%EC%A3%BC%EC%B0%A8%20%EA%B8%B0%EB%B3%B8%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
pip install langchain-community langchain-chroma langchain-openai bs4

In [51]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [52]:
from google.colab import userdata

api_key = userdata.get('OPENAI_API_KEY')

llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

In [53]:
loader = WebBaseLoader(
    web_paths=("https://spartacodingclub.kr/blog/all-in-challenge_winner",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("editedContent")
        )
    ),
)
docs = loader.load()
print(docs)

[Document(metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content='ì½”ë”©ì�€ ë�” ì�´ìƒ� ê°œë°œì��ë§Œì�˜ ì˜�ì—\xadì�´ ì•„ë‹™ë‹ˆë‹¤. ëˆ„êµ¬ë‚˜ ì•„ì�´ë””ì–´ë§Œ ì�ˆë‹¤ë©´ ì°½ì�˜ì\xa0�ì�¸ ì„œë¹„ìŠ¤ë¥¼ ë§Œë“¤ì–´ ì„¸ìƒ�ì�„ ë°”ê¿€ ìˆ˜ ì�ˆìŠµë‹ˆë‹¤. ìŠ¤íŒŒë¥´íƒ€ì½”ë”©í�´ëŸ½ì—�ì„œëŠ” ì�´ëŸ¬í•œ ê°€ëŠ¥ì„±ì�„ ë¯¿ê³\xa0, ëˆ„êµ¬ë‚˜ ì½”ë”©ì�„ í†µí•´ ì��ì‹\xa0ì�˜ ì•„ì�´ë””ì–´ë¥¼ ì‹¤í˜„í•˜ê³\xa0 ì‹¤ì\xa0œ ë¬¸ì\xa0œë¥¼ í•´ê²°í•˜ëŠ” ê²½í—˜ì�„ ìŒ“ì�„ ìˆ˜ ì�ˆë�„ë¡� ë‹¤ì–‘í•œ í”„ë¡œê·¸ë�¨ì�„ ë§ˆë\xa0¨í•˜ê³\xa0 ì�ˆìŠµë‹ˆë‹¤.<All-in> ì½”ë”© ê³µëª¨ì\xa0„ì�€ ëŒ€í•™ìƒ�ë“¤ì�´ ìº\xa0í�¼ìŠ¤ì—�ì„œ ê²ªì�€ ë¶ˆí�¸í•¨ê³¼ ë¬¸ì\xa0œë¥¼ ì��ì‹\xa0ë§Œì�˜ ì•„ì�´ë””ì–´ë¡œ í•´ê²°í•´ë³´ëŠ” ëŒ€íšŒì˜€ëŠ”ë�°ìš”. ì�´ë²ˆ ê³µëª¨ì\xa0„ì—�ì„œ ë‹¤ì–‘í•œ í˜�ì‹\xa0ì\xa0�ì�¸ ì•„ì�´ë””ì–´ì™€ ì—´ì\xa0•ìœ¼ë¡œ ê°€ë“�í•œ ìˆ˜ë§�ì�€ í”„ë¡œì\xa0�íŠ¸ê°€ íƒ„ìƒ�í–ˆìŠµë‹ˆë‹¤. ê·¸ì¤‘ ë›°ì–´ë‚œ ì„±ê³¼ë¥¼ ë‚¸ ìˆ˜ìƒ�ì�‘ 6ê°œë¥¼ ì†Œê°œí•©ë‹ˆë‹¤.ğŸ�†Â\xa0ëŒ€ìƒ�[Lexi Note] ì–¸ì–´ê³µë¶€ í•„ê¸° ì›¹ ì„œë¹„ìŠ¤ì„œë¹„ìŠ¤ ì\xa0œì

In [54]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=api_key)
)

In [55]:
retriever = vectorstore.as_retriever()

user_msg = """제시한 블로그는 All-in 코딩 공모전 수상작 소개하는 블로그야. 여기서 각 수장작별로 3줄 요약 진행해줘"""
retrieved_docs = retriever.invoke(user_msg)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
prompt = hub.pull("rlm/rag-prompt")

user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(user_prompt)

messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: 제시한 블로그는 All-in 코딩 공모전 수상작 소개하는 블로그야. 여기서 각 수장작별로 3줄 요약 진행해줘 \nContext: Spring Boot, MySQL, Spring WebSocket-BE(ë°±ì—”ë“œ): React Native, TanStack Query, Axiosì½”ë”© ê³µëª¨ì\xa0„ ìˆ˜ìƒ�ì�‘ì�€ ëŒ€í•™ìƒ�ë“¤ì�˜ íŒ€í”„ë¡œì\xa0�íŠ¸ë¥¼ í†µí•´ ë§Œë“¤ì–´ì§„ ì›¹/ì•± ì„œë¹„ìŠ¤ì�…ë‹ˆë‹¤. ìº\xa0í�¼ìŠ¤ì—�ì„œì�˜ ë¬¸ì\xa0œë¥¼ í•´ê²°í•˜ì��ëŠ” ì°¸ê°€ì��ë“¤ì�˜ ì•„ì�´ë””ì–´ì—�ì„œ ì‹œì�‘ë�˜ì—ˆì£\xa0. ëˆ„êµ¬ë‚˜ ì„¸ìƒ�ì—� ì„\xa0ë³´ì�´ê³\xa0 ì‹¶ì�€ ë‚˜ë§Œì�˜ ì•„ì�´ë””ì–´ì™€ ê¸°ì´ˆ ì½”ë”© ê¸°ìˆ\xa0ë§Œ í™œìš©í•œë‹¤ë©´, ì–¼ë§ˆë“\xa0ì§€ ì„œë¹„ìŠ¤ë¥¼ ë§Œë“¤ ìˆ˜ ì�ˆìŠµë‹ˆë‹¤. ìŠ¤íŒŒë¥´íƒ€ì½”ë”©í�´ëŸ½ì�˜ ë‚´ì�¼ë°°ì›€ìº\xa0í”„ì—�ì„œëŠ” ë¹„ì\xa0„ê³µ, ì´ˆë³´ì��ë�„ ì›¹/ì•± ê°œë°œì��ë¡œ ê±°ë“\xadë‚\xa0 ìˆ˜ ì�ˆëŠ” ë‹¤ì–‘í•œ íŠ¸ë�™ì�´ ì¤€ë¹„ë�¼ ì

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [56]:
response = llm.invoke(user_prompt)
print(response.content)

이 블로그는 All-in 코딩 공모전 수상작들을 소개합니다. 각 수상작은 Spring Boot, MySQL, Spring WebSocket을 기반으로 하며, React Native와 TanStack Query를 활용하여 사용자 경험을 향상시키는 데 중점을 두고 있습니다. 또한, 데이터 처리와 효율적인 상호 작용을 위한 Axios 라이브러리를 포함하여, 현대적인 웹 애플리케이션 개발에 적합한 기술 스택을 강조합니다.


# 첫번째 프롬프트
제시한 블로그는 All-in 코딩 공모전 수상작 소개하는 블로그야. 여기서 각 수장작별로 3줄 요약 진행해줘


답변 :

1. **Spring Boot, MySQL, Spring WebSocket**: 이 프로젝트는 실시간 데이터 전송을 위한 웹소켓 기능을 활용하여 React Native 앱에서 데이터를 동적으로 업데이트할 수 있도록 구현되었습니다. 클라이언트와 서버 간의 긴밀한 통신을 통해 사용자 경험을 향상시키는 것을 목표로 하고 있습니다. 데이터베이스와의 연동을 통해 사용자의 요청을 효율적으로 처리하고 있습니다.

2. **React, Tesseract.js, React-Quill**: 이 application은 사용자들이 이미지를 텍스트로 변환하고 편집할 수 있는 기능을 제공하며, Tesseract.js를 활용하여 OCR(Optical Character Recognition)을 구현합니다. 사용자 인터페이스는 React와 React-Quill을 이용해 직관적으로 설계되었으며, 사용자 편의를 고려한 다양한 기능이 포함되어 있습니다. 데이터는 효과적으로 관리 및 저장되며, 이는 사용자에게 더 나은 경험을 제공합니다.

3. **React Native, Expo, Axios**: 이 프로젝트는 모바일 앱 개발을 위한 프레임워크인 React Native를 사용하여, 다양한 API와 연동하는 앱을 구현하였습니다. 사용자 인증 및 데이터 전송을 위한 Spring Boot와 MySQL 백엔드와 연결되어 있으며, Expo를 통한 빠른 개발이 장점입니다. Axios를 활용하여 네트워크 요청과 응답 처리를 간편하게 수행합니다.




# 두번째 프롬프트
